## Set up the drive path

In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [1]:
ls

datalab/  drive/


In [2]:
cd drive/RNNSearch

/content/drive/RNNSearch


## Import Library

In [0]:
import collections
import os
import sys

import tensorflow as tf
import numpy as np

from tensorflow.contrib.seq2seq import sequence_loss

import math
import re
import matplotlib.pyplot as plt

import pickle

!pip install -q mosestokenizer
from mosestokenizer import *

from nltk.translate.bleu_score import sentence_bleu

## Data Pre-processing

In [0]:
## load vocab dict from txt file

f = open("../dictionary/en_word_to_id.txt", "rb")
en_word_to_id = pickle.load(f)
f.close()

f = open("../dictionary/fr_word_to_id.txt", "rb")
fr_word_to_id = pickle.load(f)
f.close()

In [7]:
en_vocab_size = len(en_word_to_id)

fr_word_to_id['<beg>'] = len(fr_word_to_id)
fr_vocab_size = len(fr_word_to_id)

print(en_vocab_size)
print(fr_vocab_size)

30772
39579


In [0]:
def _read_words(filename):
  with tf.gfile.GFile(filename, "r") as f: 
    output = f.read().replace("\n", " eos ").replace(".", " .")
    output = re.sub('[0-9]+', 'N', output)
    return output

def _file_to_word_ids(data, word_to_id):
  
  id_list = []
  
  for word in data:
    if word in word_to_id:
      id_list.append(word_to_id[word])
    else:
      id_list.append(1)
          
  return id_list


def preprocess_train_data(pre_data, word_to_id, max_length):
    pre_data_array = np.asarray(pre_data)
    last_start = 0
    data = []
    each_sen_len = []
    
    for i in range(len(pre_data_array)):
        if pre_data_array[i]==word_to_id['eos']:
            if max_length >= len(pre_data_array[last_start:(i+1)]):                
              data.append(pre_data_array[last_start:(i+1)])
              each_sen_len.append(i+1-last_start)
              
            else:
              shorten_sentences = pre_data_array[last_start:(last_start+max_length-1)]
              shorten_sentences = np.concatenate((shorten_sentences, np.asarray([word_to_id['eos']])), axis=0)
              data.append(shorten_sentences)
              each_sen_len.append(max_length) 
            
            last_start = i+1
            
    out_sentences = np.full([len(data), max_length], word_to_id['<PAD>'], dtype=np.int32)
    for i in range(len(data)):
        out_sentences[i,:len(data[i])] = data[i]    
    return out_sentences, np.asarray(each_sen_len)
            
    return data, each_sen_len

In [0]:
def generate_input_en(en_file, en_word_to_id, max_length):
  
    en_data = _read_words(en_file)

    en_tokenize = MosesTokenizer('en')

    en_data = en_tokenize(en_data)

    en_data_id = _file_to_word_ids(en_data, en_word_to_id)

    en_input, en_input_len = preprocess_train_data(en_data_id, en_word_to_id, max_length)
    
    return en_input, en_input_len
  
  
  
def generate_output_fr(fr_file, fr_word_to_id, max_length):
    
    fr_data = _read_words(fr_file)

    fr_tokenize = MosesTokenizer('fr')

    fr_data = fr_tokenize(fr_data)

    fr_data_id = _file_to_word_ids(fr_data, fr_word_to_id)

    fr_output, fr_output_len = preprocess_train_data(fr_data_id, fr_word_to_id,max_length)

    out_beg_token = fr_word_to_id['<beg>']*np.ones((fr_output.shape[0], 1), dtype=np.int32)

    fr_output = np.concatenate((out_beg_token, fr_output), axis=1)

    return fr_output,fr_output_len

In [0]:
def batch_producer(raw_data, raw_data_len, batch_size):    
    data_len = len(raw_data)    
    batch_len = data_len // batch_size    
    data = np.reshape(raw_data[0 : batch_size * batch_len, :], [batch_size, batch_len, -1])
    data = np.transpose(data, (1,0,2))
    
    data_length = np.reshape(raw_data_len[0 : batch_size * batch_len], [batch_size, batch_len])
    data_length = np.transpose(data_length, (1,0))
    return data, data_length 

## Model

In [0]:
###################### define parameters ######################

max_length = 40     # max sentence length

batch_size = 50

embed_size = 300

hidden_size = 500

align_size = 500

maxout_size = 500

learning_rate = 0.001

regular_rate = 0.00001

In [0]:
###################### define placeholder ######################

input_placeholder = tf.placeholder(tf.int32, [None, max_length], 'input')                   # batch_size x en_max_length

target_placeholder = tf.placeholder(tf.int32, [None, max_length+1], 'target')                 # batch_size x fr_max_length

in_length_placeholder = tf.placeholder(tf.int32, [None, ], 'in_len')                         # batch_size x 1

out_length_placeholder = tf.placeholder(tf.int32, [None, ], 'out_len')                       # batch_size x 1

xavier_initializer = tf.contrib.layers.xavier_initializer()

In [0]:
##################### embedding look-up for input sentences ######################

with tf.variable_scope('en_embedding'):
    en_embedding = tf.get_variable('en_embeding',[en_vocab_size, embed_size], dtype=tf.float32, initializer=xavier_initializer)
    inputs = tf.nn.embedding_lookup(en_embedding, input_placeholder)                                      # batch_size x en_max_length x embed_size
    

with tf.variable_scope('fr_embedding'):
    fr_embedding = tf.get_variable('fr_embeding',[fr_vocab_size, embed_size], dtype=tf.float32, initializer=xavier_initializer)
    targets = tf.nn.embedding_lookup(fr_embedding, target_placeholder)                                    # batch_size x fr_max_length x embed_size

In [14]:
##################### encoder ######################
##################### bi-direction lstm ######################

encode_inputs = tf.transpose(inputs, (1,0,2))                                   # en_max_length x batch_size x embed_size
  
with tf.variable_scope('encode'):
    basic_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0, state_is_tuple=True)
    init_state = basic_cell.zero_state(batch_size, tf.float32)
    encode_outputs, encode_state = tf.nn.bidirectional_dynamic_rnn(cell_fw=basic_cell, cell_bw=basic_cell, inputs=encode_inputs,
                                                                   sequence_length=in_length_placeholder,
                                                                   initial_state_fw=init_state,
                                                                   initial_state_bw=init_state,
                                                                   dtype=tf.float32,
                                                                   time_major=True)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [0]:
##################### handle the final state of encoder ######################

fw_bw_en_outputs = tf.concat((encode_outputs[0],encode_outputs[1]),2)           # en_max_length x batch_size x 2*hidden_size
fw_bw_en_outputs_list = tf.split(fw_bw_en_outputs, max_length, axis=0)       # en_max_length x batch_size x 2*hidden_size

annotations = tf.transpose(fw_bw_en_outputs,(1,0,2))                                      # batch_size x en_max_length x 2*hidden_size
annotations_reshaped = tf.reshape(annotations, (batch_size*max_length, 2*hidden_size)) # batch_size*en_max_length x 2*hidden_size

In [0]:
##################### parameters ######################

with tf.variable_scope('alignment'):
    W_a = tf.get_variable('W_a',[hidden_size, align_size], dtype=tf.float32, initializer=xavier_initializer)
    U_a = tf.get_variable('U_a',[2*hidden_size, align_size], dtype=tf.float32, initializer=xavier_initializer)
    v_a = tf.get_variable('v_a',[align_size,1], dtype=tf.float32, initializer=xavier_initializer)
      

with tf.variable_scope('decoder'):
    W_s_init = tf.get_variable('W_s_init',[hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    
    W_z = tf.get_variable('W_z',[embed_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    U_z = tf.get_variable('U_z',[hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    C_z = tf.get_variable('C_z',[2*hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    
    W_r = tf.get_variable('W_r',[embed_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    U_r = tf.get_variable('U_r',[hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    C_r = tf.get_variable('C_r',[2*hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    
    W_ss = tf.get_variable('W_ss',[embed_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    U_ss = tf.get_variable('U_ss',[hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    C_ss = tf.get_variable('C_ss',[2*hidden_size, hidden_size], dtype=tf.float32, initializer=xavier_initializer)
    
    
with tf.variable_scope('probability'):
    W_o = tf.get_variable('W_o',[maxout_size, embed_size], dtype=tf.float32, initializer=xavier_initializer)
    U_o = tf.get_variable('U_o',[hidden_size, 2*maxout_size], dtype=tf.float32, initializer=xavier_initializer)
    V_o = tf.get_variable('V_o',[embed_size,  2*maxout_size], dtype=tf.float32, initializer=xavier_initializer)
    C_o = tf.get_variable('C_o',[2*hidden_size, 2*maxout_size], dtype=tf.float32, initializer=xavier_initializer)

In [0]:
UaHj = tf.matmul(annotations_reshaped, U_a)

In [0]:
##################### decoder ######################

#### define the inital hidden state for decoder
s_0 = tf.nn.tanh(tf.matmul(encode_state[1].h, W_s_init))                        # batch_size x hidden_size
prob_norm = []
prob_constant = []

#### decoder loop ####

for i in range(max_length):
    
    #### alignment model
    WaS0 = tf.matmul(s_0, W_a)                                                  # batch_size x align_size
    WaS0_allsteps = tf.tile(WaS0, [max_length, 1])                        # batch_size*en_max_length x align_size
    
    e_1 = tf.matmul(tf.nn.tanh(WaS0_allsteps + UaHj), v_a)                      # batch_size*en_max_length x 1
    e_1j = tf.reshape(e_1, [batch_size,max_length])                        # batch_size x en_max_length
    
    alpha_1 = tf.nn.softmax(e_1j)                                               # batch_size x en_max_length
    alpha_1_reshaped = tf.reshape(alpha_1,(batch_size*max_length,1))         # batch_size*en_max_length x 1
    
    
    #### context vector
    c_1j = tf.reshape(annotations_reshaped*alpha_1_reshaped, (batch_size, max_length, 2*hidden_size))  # batch_size x en_max_length x 2*hidden_size
    c_1 = tf.reduce_sum(c_1j, axis=1)                                                                     # batch_size x 2*hidden_size
    
    
    #### decoder hidden variables
    r_1 = tf.nn.sigmoid(tf.matmul(targets[:,i,:], W_r) + tf.matmul(s_0, U_r) + tf.matmul(c_1, C_r))               # batch_size x hidden_size

    z_1 = tf.nn.sigmoid(tf.matmul(targets[:,i,:], W_z) + tf.matmul(s_0, U_z) + tf.matmul(c_1, C_z))               # batch_size x hidden_size

    ss_1 = tf.nn.tanh(tf.matmul(targets[:,i,:], W_ss) + tf.matmul(r_1*s_0, U_ss) + tf.matmul(c_1, C_ss))          # batch_size x hidden_size
    
    s_1 = (1-z_1)*s_0 + z_1*ss_1                                                                                  # batch_size x hidden_size
    
    
    #### probability of the target word
    tt_1 = tf.matmul(s_0, U_o) + tf.matmul(targets[:,i,:], V_o) + tf.matmul(c_1, C_o)       # batch_size x 2*maxout_size
    t_1 = tf.contrib.layers.maxout(tt_1,maxout_size,axis=-1)                                # batch_size x maxout_size
    
    WoT1 = tf.matmul(t_1, W_o)
    prob_unnorm_1 = tf.exp(tf.reduce_sum(targets[:,i+1,:]*WoT1, axis=1))                      # batch_size x 1
    
    prob_constant_1 = tf.exp(tf.matmul(WoT1, tf.transpose(fr_embedding,(1,0))))               # batch_size x fr_vocab_size
    
    prob_norm_1 = prob_unnorm_1/tf.reduce_sum(prob_constant_1, axis=1)                            # batch_size x 1
    
    prob_constant.append(prob_constant_1)

    prob_norm.append(prob_norm_1)
    
    
    #### update the hidden state
    s_0 = s_1
 
  

In [0]:
log_prob_norm = tf.log(tf.stack(prob_norm, axis=1))

out_seq_weight = tf.sequence_mask(out_length_placeholder, maxlen=max_length, dtype=tf.float32)

nll = - tf.reduce_sum(log_prob_norm*out_seq_weight, axis=1)

nll_average = tf.reduce_mean(nll)

In [0]:
train_variables = tf.trainable_variables()

regularization_cost = tf.reduce_sum([tf.nn.l2_loss(variable) for variable in train_variables])

objective = nll_average + regular_rate*regularization_cost

In [0]:
# optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001, rho=0.95, epsilon=1e-6)
# train_op = optimizer.minimize(nll_average)

opt = tf.train.AdamOptimizer(learning_rate).minimize(objective)

In [0]:
#### save the model
def save_model(session, path):
    if not os.path.exists("./result_4/"):
        os.mkdir('./result_4/')
    saver = tf.train.Saver()
    saver.save(session, path)

path1 = './result_4/model_each_epch.ckpt'

## Training 

In [0]:
file_en_test = "../translation_test_data/newstest2012.en"

file_fr_test = "../translation_test_data/newstest2012.fr"

en_test, en_test_len = generate_input_en(file_en_test, en_word_to_id, max_length)

fr_test, fr_test_len = generate_output_fr(file_fr_test, fr_word_to_id, max_length)

file_en_test = "../translation_test_data/newstest2013.en"

file_fr_test = "../translation_test_data/newstest2013.fr"

en_test_2, en_test_len_2 = generate_input_en(file_en_test, en_word_to_id, max_length)

fr_test_2, fr_test_len_2 = generate_output_fr(file_fr_test, fr_word_to_id, max_length)

In [0]:
en_test = np.concatenate((en_test,en_test_2),axis=0)
en_test_len = np.concatenate((en_test_len,en_test_len_2),axis=0)

fr_test = np.concatenate((fr_test,fr_test_2),axis=0)
fr_test_len = np.concatenate((fr_test_len,fr_test_len_2),axis=0)

en_test_batches, en_test_len_batches = batch_producer(en_test, en_test_len, batch_size) 
fr_test_batches, fr_test_len_batches = batch_producer(fr_test, fr_test_len, batch_size)

In [0]:
max_epochs = 1

nll_aver_results=[]
nll_aver_dep_results = []

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    for epoc in range(max_epochs):
      
        print('Epoch {}'.format(epoc))

        ind_small_txt = epoc
        
        en_file = "../small_txt/" + str(ind_small_txt) + "_en.txt"
        fr_file = "../small_txt/" + str(ind_small_txt) + "_fr.txt"
        
        print(en_file)
        print(fr_file)
        
        en_input, en_input_len = generate_input_en(en_file, en_word_to_id, max_length)
        fr_output, fr_output_len = generate_output_fr(fr_file, fr_word_to_id, max_length)

        en_input_batches, en_input_len_batches = batch_producer(en_input, en_input_len, batch_size) 
        fr_output_batches, fr_output_len_batches = batch_producer(fr_output, fr_output_len, batch_size)
        
        batch_len = en_input_batches.shape[0]
      
        ########### training ###########
        for i in range(batch_len):
        
 
            feed_dict = {input_placeholder: en_input_batches[i], 
                         target_placeholder: fr_output_batches[i],
                         in_length_placeholder: en_input_len_batches[i], 
                         out_length_placeholder: fr_output_len_batches[i]}
      
            nll_aver, regu_cost, _ = sess.run([nll_average,regularization_cost,opt], feed_dict=feed_dict)
      
            if i%100 == 0:
                nll_aver_results.append(nll_aver)
                print(nll_aver)
                #print(regu_cost)
        
            if i%1000 == 0:
                save_model(sess, path1)
              
                test_nll = []
                for h in range(en_test_batches.shape[0]):
                  
                  test_feed_dict = {input_placeholder: en_test_batches[h], 
                                    target_placeholder: fr_test_batches[h],
                                    in_length_placeholder: en_test_len_batches[h], 
                                    out_length_placeholder: fr_test_len_batches[h]}
                  
                  test_nll_aver = sess.run(nll_average, feed_dict=test_feed_dict)
                  test_nll.append(test_nll_aver)
                
                print("tesing result: updated every 1000 updates")
                print(np.mean(test_nll))
                nll_aver_dep_results.append(np.mean(test_nll))
                  
              
                

In [32]:
max_epochs = 2
nll_aver_results = []
nll_aver_dep_results = []

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, path1)
    
    for epoc in range(max_epochs):
      
        print('Epoch {}'.format(epoc))

        ind_small_txt = epoc + 1
        
        en_file = "./small_txt/" + str(ind_small_txt) + "_en.txt"
        fr_file = "./small_txt/" + str(ind_small_txt) + "_fr.txt"
        
        print(en_file)
        print(fr_file)
        
        en_input, en_input_len = generate_input_en(en_file, en_word_to_id, max_length)
        fr_output, fr_output_len = generate_output_fr(fr_file, fr_word_to_id, max_length)

        en_input_batches, en_input_len_batches = batch_producer(en_input, en_input_len, batch_size) 
        fr_output_batches, fr_output_len_batches = batch_producer(fr_output, fr_output_len, batch_size)
        
        batch_len = en_input_batches.shape[0]
        
        ########### training ###########
        for i in range(batch_len):
        
 
            feed_dict = {input_placeholder: en_input_batches[i], 
                         target_placeholder: fr_output_batches[i],
                         in_length_placeholder: en_input_len_batches[i], 
                         out_length_placeholder: fr_output_len_batches[i]}
      
            nll_aver, _ = sess.run([nll_average,opt], feed_dict=feed_dict)
      
            if i%100 == 0:
                nll_aver_results.append(nll_aver)
                print(nll_aver)
            
            if i%1000 == 0:
                save_model(sess, path1)
              
                test_nll = []
                for h in range(en_test_batches.shape[0]):
                  
                  test_feed_dict = {input_placeholder: en_test_batches[h], 
                                    target_placeholder: fr_test_batches[h],
                                    in_length_placeholder: en_test_len_batches[h], 
                                    out_length_placeholder: fr_test_len_batches[h]}
                  
                  test_nll_aver = sess.run(nll_average, feed_dict=test_feed_dict)
                  test_nll.append(test_nll_aver)
                
                print("tesing result: updated every 1000 updates")
                print(np.mean(test_nll))
                nll_aver_dep_results.append(np.mean(test_nll))
            

INFO:tensorflow:Restoring parameters from ./result_4/model_each_epch.ckpt
Epoch 0
./small_txt/1_en.txt
./small_txt/1_fr.txt
109.3119
tesing result: updated every 1000 updates
123.16238
103.4042
102.48878
108.78156
84.672424
100.86406
100.90387
97.66413
80.823746
92.261345
91.72873
tesing result: updated every 1000 updates
112.958984
92.79151
87.88254
88.835175
91.120316
91.90158
93.43369
100.87776
81.57022
93.91861
89.25955
tesing result: updated every 1000 updates
106.317345
75.979355
74.57379
91.66147
94.33628
87.79422
64.76533
87.8458
67.85983
97.79245
Epoch 1
./small_txt/2_en.txt
./small_txt/2_fr.txt
89.917755
tesing result: updated every 1000 updates
103.07902
85.867256
78.4876
73.660675
84.77483
84.61196
63.287315
86.49585
83.71025
72.51001
82.783005
tesing result: updated every 1000 updates
97.94598
76.75386
79.65358
74.85089
70.279274
79.21473
72.214226
81.79712
73.43077
72.368866
80.73326
tesing result: updated every 1000 updates
96.30542
77.83921
72.02315
69.0442
59.855392
71

In [37]:
len(nll_aver_results)

60

In [0]:
def text_save(content,filename,mode='a'):
    # Try to save a list variable in txt file.
    file = open(filename,mode)
    for i in range(len(content)):
        file.write(str(content[i])+'\n')
    file.close()
    
def text_read(filename):
    # Try to read a txt file and return a list.Return [] if there was a mistake.
    try:
        file = open(filename,'r')
    except IOError:
        error = []
        return error
    content = file.readlines()
 
    for i in range(len(content)):
        content[i] = content[i][:len(content[i])-1]
 
    file.close()
    return content

In [0]:
text_save(nll_aver_results,'./nll_aver_result_v4_epoch_1.txt')

In [0]:
read_results = text_read('./nll_aver_result_v4_epoch_1.txt')

In [36]:
len(read_results) #012

90

In [0]:
######## test set ########

nll_aver_test_results = []

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, path1)
    
    for i in range(en_test_batches.shape[0]):
      
      test_feed_dict = {input_placeholder: en_test_batches[i], 
                        target_placeholder: fr_test_batches[i],
                        in_length_placeholder: en_test_len_batches[i], 
                        out_length_placeholder: fr_test_len_batches[i]}
      
      nll_aver_test = sess.run(nll_average, feed_dict=test_feed_dict)
      
      nll_aver_test_results.append(nll_aver_test)
    
    print(np.mean(nll_aver_test_results))
      

INFO:tensorflow:Restoring parameters from ./result_2/model_each_epch.ckpt
76.7958


In [0]:
ind_small_txt = 12
        
en_file = "./small_txt/" + str(ind_small_txt) + "_en.txt"
fr_file = "./small_txt/" + str(ind_small_txt) + "_fr.txt"
        
print(en_file)
print(fr_file)

en_input, en_input_len = generate_input_en(en_file, en_word_to_id, max_length)
fr_output, fr_output_len = generate_output_fr(fr_file, fr_word_to_id, max_length)

en_input_batches, en_input_len_batches = batch_producer(en_input, en_input_len, batch_size) 
fr_output_batches, fr_output_len_batches = batch_producer(fr_output, fr_output_len, batch_size)
        
batch_len = en_input_batches.shape[0]

./small_txt/12_en.txt
./small_txt/12_fr.txt


In [0]:
######## train set (the last txt) ########

nll_aver_subtrain_results = []

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, path1)
    
    for i in range(batch_len):
 
      train_feed_dict = {input_placeholder: en_input_batches[i], 
                         target_placeholder: fr_output_batches[i],
                         in_length_placeholder: en_input_len_batches[i], 
                         out_length_placeholder: fr_output_len_batches[i]}
      
      nll_aver_train = sess.run(nll_average, feed_dict=train_feed_dict)
   
      nll_aver_subtrain_results.append(nll_aver_train)
    
    print(np.mean(nll_aver_subtrain_results))
      

INFO:tensorflow:Restoring parameters from ./result_2/model_each_epch.ckpt
61.65899


## Translation Samples 

### Select the next target word with highest probability (Greedy Search)

In [0]:
with tf.Session() as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, path1)
    
    fr_test_translated = np.copy(fr_test_batches)
    
    for k in range(6):      
      for i in range(max_length):
        prob_next_word = np.zeros((batch_size, fr_vocab_size))
      
        test_feed_dict = {input_placeholder: en_test_batches[k], 
                          target_placeholder: fr_test_translated[k],
                          in_length_placeholder: en_test_len_batches[k], 
                          out_length_placeholder: fr_test_len_batches[k]}
      
      
        prob_all_words = sess.run(prob_constant, feed_dict=test_feed_dict)
      
        for j in range(batch_size):
          prob_next_word[j] = prob_all_words[i][j]/np.sum(prob_all_words[i][j])
          fr_test_translated[k, j,i+1] = np.argmax(prob_next_word[j])

INFO:tensorflow:Restoring parameters from ./result_2/model_each_epch.ckpt


In [0]:
def id_to_word(words, word_to_id, max_length):
  sens = ["" for x in range(max_length+1)]
  for key in word_to_id.keys():
    for p in range(max_length+1):
      if words[p] == word_to_id[key]:
        sens[p] = key
  return sens

In [0]:
total_scores = []
for k in range(3):
  for i in range(batch_size):  
    gene_sen = id_to_word(fr_test_translated[k,i], fr_word_to_id,max_length)
    orig_sen = id_to_word(fr_test_batches[k,i], fr_word_to_id,max_length)

    score = sentence_bleu(orig_sen, gene_sen)  #(reference, candidates)
    total_scores.append(score)
    
print(np.mean(total_scores))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3134934988955799


In [0]:
id_sen = 9

en_sens_str = ""

en_sens = id_to_word(en_test_batches[0,id_sen], en_word_to_id, max_length-1)
for p in range(len(en_sens)):
        en_sens_str = en_sens_str + " " + en_sens[p]
    
print(en_sens_str)


fr_sens_str = ""

fr_sens = id_to_word(fr_test_batches[0,id_sen], fr_word_to_id, max_length)
for p in range(len(fr_sens)):
        fr_sens_str = fr_sens_str + " " + fr_sens[p]
    
print(fr_sens_str)

tr_sens_str = ""

tr_sens = id_to_word(fr_test_translated[0,id_sen], fr_word_to_id, max_length)
for p in range(len(tr_sens)):
        tr_sens_str = tr_sens_str + " " + tr_sens[p]
    
print(tr_sens_str)

print(sentence_bleu(fr_sens_str, tr_sens_str))

 The Croatians continued in fine form after half @-@ time . eos <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
 <beg> Même après la pause , les Croates n&apos; ont rien laissé brûler . eos <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
 <beg> Les Croates ont continué à la valeur de la durée de la période de longue période . eos mois de janvier , la Commission a un autre point de vue
0.6127146389227656


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
